In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [2]:
base_model = "NousResearch/Llama-2-7b-chat-hf"


In [3]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/chowder/.local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/chowder/.local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/chowder/.local/lib/python3.8/site-packages/transform

In [5]:
# Load LLaMA tokenizer
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [6]:
prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=base_model, max_length=50)
#result = pipe(f"<s>[INST] {prompt} [/INST]")
#print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
prompt = "how are you doing?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])
print(result)

<s>[INST] how are you doing? [/INST]  I'm just an AI, I don't have feelings or emotions like humans do, so I don't have a physical or mental state that can be described
[{'generated_text': "<s>[INST] how are you doing? [/INST]  I'm just an AI, I don't have feelings or emotions like humans do, so I don't have a physical or mental state that can be described"}]
